In [1]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Thu Nov 16 20:40:27 2023       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 536.67                 Driver Version: 536.67       CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                     TCC/WDDM  | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA GeForce GTX 1080      WDDM  | 00000000:26:00.0  On |                  N/A |
|  0%   34C    P0              33W / 180W |    694MiB /  8192MiB |      1%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [2]:
!pip install datasets
!pip install git+https://github.com/huggingface/transformers
!pip install librosa
!pip install evaluate
!pip install jiwer
!pip install gradio
!pip install accelerate


  Cloning https://github.com/huggingface/transformers to c:\users\ondre\appdata\local\temp\pip-req-build-6yj5n7o6
  Resolved https://github.com/huggingface/transformers to commit 12b50c6130ccfbb2381a181dc4bbb2bf07a3c62a
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
INFO: pip is looking at multiple versions of tokenizers to determine which version is compatible with other requirements. This could take a while.
   ---------------------------------------- 0.0/311.7 kB ? eta -:--:--
   ----------- ---------------------------- 92.2/311.7 kB 1.7 MB/s eta 0:00:01
   ---------------------- ----------------- 174.1/311.7 kB 2.1 MB/s eta 0:00:01
   ---------------------------------------- 311.7/311.7 kB 2.4 MB/s eta 

  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/transformers 'C:\Users\ondre\AppData\Local\Temp\pip-req-build-6yj5n7o6'
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
faster-whisper 0.6.0 requires tokenizers==0.13.*, but you have tokenizers 0.15.0 which is incompatible.


In [25]:
from huggingface_hub import notebook_login

notebook_login()

In [5]:
from datasets import load_dataset, ClassLabel

dataset = load_dataset("speech_commands", "v0.02", verification_mode = "no_checks")

In [6]:
dataset = dataset.select_columns(["audio", "label"])

In [7]:
labels = dataset["train"].features["label"].names
label2id, id2label = dict(), dict()
for i, label in enumerate(labels):
    label2id[label] = str(i)
    id2label[str(i)] = label

id2label["0"]

'yes'

In [8]:
selected_classes = ["no", "go"]
dataset = dataset.filter(lambda example: id2label[str(example['label'])] in selected_classes)

In [9]:
print(dataset)

DatasetDict({
    train: Dataset({
        features: ['audio', 'label'],
        num_rows: 6236
    })
    validation: Dataset({
        features: ['audio', 'label'],
        num_rows: 778
    })
    test: Dataset({
        features: ['audio', 'label'],
        num_rows: 807
    })
})


In [10]:
dataset["train"]["label"][20]

9

In [11]:
from transformers import AutoFeatureExtractor

model_id = "facebook/wav2vec2-base-960h"
#model_id = "ntu-spml/distilhubert"
feature_extractor = AutoFeatureExtractor.from_pretrained(
    model_id, do_normalize=True, return_attention_mask=True
)

(…)0h/resolve/main/preprocessor_config.json:   0%|          | 0.00/159 [00:00<?, ?B/s]

(…)2vec2-base-960h/resolve/main/config.json:   0%|          | 0.00/1.60k [00:00<?, ?B/s]

In [12]:
sampling_rate = feature_extractor.sampling_rate
sampling_rate

16000

In [13]:
from datasets import Audio

dataset = dataset.cast_column("audio", Audio(sampling_rate=sampling_rate))

## Nomalizetion

In [14]:
import numpy as np

sample = dataset["train"][0]["audio"]

print(f"Mean: {np.mean(sample['array']):.3}, Variance: {np.var(sample['array']):.3}")

Mean: 2.65e-06, Variance: 0.00253


In [ ]:
inputs = feature_extractor(sample["array"], sampling_rate=sample["sampling_rate"])

print(f"inputs keys: {list(inputs.keys())}")

print(
    f"Mean: {np.mean(inputs['input_values']):.3}, Variance: {np.var(inputs['input_values']):.3}"
)

## Preprocess

In [14]:
max_duration = 2

def preprocess_function(examples):
    audio_arrays = [x["array"] for x in examples["audio"]]
    inputs = feature_extractor(
        audio_arrays, 
        sampling_rate=feature_extractor.sampling_rate, 
        max_length=int(feature_extractor.sampling_rate * max_duration), 
        truncation=True,
        return_attention_mask=True,
    )
    return inputs

In [15]:
dataset_encoded = dataset.map(
    preprocess_function,
    remove_columns="audio",
    batched=True,
    batch_size=100,
    num_proc=1,
)
dataset_encoded

Map:   0%|          | 0/6236 [00:00<?, ? examples/s]

Map:   0%|          | 0/778 [00:00<?, ? examples/s]

Map:   0%|          | 0/807 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['label', 'input_values', 'attention_mask'],
        num_rows: 6236
    })
    validation: Dataset({
        features: ['label', 'input_values', 'attention_mask'],
        num_rows: 778
    })
    test: Dataset({
        features: ['label', 'input_values', 'attention_mask'],
        num_rows: 807
    })
})

In [16]:
import evaluate

accuracy = evaluate.load("accuracy")

In [17]:
import numpy as np


def compute_metrics(eval_pred):
    predictions = np.argmax(eval_pred.predictions, axis=1)
    return accuracy.compute(predictions=predictions, references=eval_pred.label_ids)

In [18]:
from transformers import AutoModelForAudioClassification, TrainingArguments, Trainer

num_labels = len(id2label)
model = AutoModelForAudioClassification.from_pretrained(
    model_id, num_labels=num_labels, label2id=label2id, id2label=id2label
)

model.safetensors:   0%|          | 0.00/378M [00:00<?, ?B/s]

Some weights of Wav2Vec2ForSequenceClassification were not initialized from the model checkpoint at facebook/wav2vec2-base-960h and are newly initialized: ['classifier.weight', 'wav2vec2.encoder.pos_conv_embed.conv.parametrizations.weight.original1', 'wav2vec2.masked_spec_embed', 'projector.bias', 'projector.weight', 'classifier.bias', 'wav2vec2.encoder.pos_conv_embed.conv.parametrizations.weight.original0']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [19]:
from transformers import TrainingArguments

model_name = model_id.split("/")[-1]
batch_size = 8
gradient_accumulation_steps = 1
num_train_epochs = 10

training_args = TrainingArguments(
    f"{model_name}-finetuned-no-go-kws",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    learning_rate=5e-5,
    per_device_train_batch_size=batch_size,
    gradient_accumulation_steps=gradient_accumulation_steps,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=num_train_epochs,
    warmup_ratio=0.1,
    logging_steps=5,
    load_best_model_at_end=True,
    metric_for_best_model="accuracy",
    fp16=True,
    push_to_hub=True,
)

In [20]:
import evaluate
import numpy as np

metric = evaluate.load("accuracy")


def compute_metrics(eval_pred):
    """Computes accuracy on a batch of predictions"""
    predictions = np.argmax(eval_pred.predictions, axis=1)
    return metric.compute(predictions=predictions, references=eval_pred.label_ids)

In [21]:
from transformers import Trainer

trainer = Trainer(
    model,
    training_args,
    train_dataset=dataset_encoded["train"].with_format("torch"),
    eval_dataset=dataset_encoded["test"].with_format("torch"),
    tokenizer=feature_extractor,
    compute_metrics=compute_metrics,
)

trainer.train()

  0%|          | 0/7800 [00:00<?, ?it/s]

{'loss': 3.5919, 'learning_rate': 3.205128205128205e-07, 'epoch': 0.01}
{'loss': 3.565, 'learning_rate': 4.487179487179487e-07, 'epoch': 0.01}
{'loss': 3.5914, 'learning_rate': 7.692307692307694e-07, 'epoch': 0.02}
{'loss': 3.5677, 'learning_rate': 1.0897435897435896e-06, 'epoch': 0.03}
{'loss': 3.5527, 'learning_rate': 1.4102564102564104e-06, 'epoch': 0.03}
{'loss': 3.5661, 'learning_rate': 1.7307692307692308e-06, 'epoch': 0.04}
{'loss': 3.5786, 'learning_rate': 2.0512820512820513e-06, 'epoch': 0.04}
{'loss': 3.5756, 'learning_rate': 2.307692307692308e-06, 'epoch': 0.05}
{'loss': 3.5702, 'learning_rate': 2.6282051282051286e-06, 'epoch': 0.06}
{'loss': 3.5614, 'learning_rate': 2.9487179487179486e-06, 'epoch': 0.06}
{'loss': 3.5568, 'learning_rate': 3.2692307692307696e-06, 'epoch': 0.07}
{'loss': 3.5447, 'learning_rate': 3.5897435897435896e-06, 'epoch': 0.08}
{'loss': 3.5363, 'learning_rate': 3.91025641025641e-06, 'epoch': 0.08}
{'loss': 3.4748, 'learning_rate': 4.230769230769231e-06, '

  0%|          | 0/101 [00:00<?, ?it/s]

{'eval_loss': 0.12290257215499878, 'eval_accuracy': 0.9727385377942999, 'eval_runtime': 4.2029, 'eval_samples_per_second': 192.011, 'eval_steps_per_second': 24.031, 'epoch': 1.0}
{'loss': 0.61, 'learning_rate': 4.9679487179487185e-05, 'epoch': 1.01}
{'loss': 0.4812, 'learning_rate': 5e-05, 'epoch': 1.01}
{'loss': 0.3969, 'learning_rate': 4.996438746438747e-05, 'epoch': 1.02}
{'loss': 0.2429, 'learning_rate': 4.992877492877493e-05, 'epoch': 1.03}
{'loss': 0.1743, 'learning_rate': 4.98931623931624e-05, 'epoch': 1.03}
{'loss': 0.4, 'learning_rate': 4.985754985754986e-05, 'epoch': 1.04}
{'loss': 0.7467, 'learning_rate': 4.982193732193732e-05, 'epoch': 1.04}
{'loss': 0.7389, 'learning_rate': 4.978632478632479e-05, 'epoch': 1.05}
{'loss': 0.1418, 'learning_rate': 4.9750712250712256e-05, 'epoch': 1.06}
{'loss': 1.592, 'learning_rate': 4.971509971509972e-05, 'epoch': 1.06}
{'loss': 0.6557, 'learning_rate': 4.9679487179487185e-05, 'epoch': 1.07}
{'loss': 0.4625, 'learning_rate': 4.9643874643874

  0%|          | 0/101 [00:00<?, ?it/s]

...

  0%|          | 0/101 [00:00<?, ?it/s]

{'eval_loss': 0.05105340853333473, 'eval_accuracy': 0.9925650557620818, 'eval_runtime': 4.2673, 'eval_samples_per_second': 189.113, 'eval_steps_per_second': 23.668, 'epoch': 9.0}
{'loss': 0.0334, 'learning_rate': 5.598290598290599e-06, 'epoch': 9.01}
{'loss': 0.5184, 'learning_rate': 5.562678062678063e-06, 'epoch': 9.01}
{'loss': 0.014, 'learning_rate': 5.527065527065527e-06, 'epoch': 9.02}
{'loss': 0.0582, 'learning_rate': 5.491452991452992e-06, 'epoch': 9.03}
{'loss': 0.0587, 'learning_rate': 5.455840455840456e-06, 'epoch': 9.03}
{'loss': 0.2042, 'learning_rate': 5.42022792022792e-06, 'epoch': 9.04}
{'loss': 0.1399, 'learning_rate': 5.3846153846153855e-06, 'epoch': 9.04}
{'loss': 0.1812, 'learning_rate': 5.34900284900285e-06, 'epoch': 9.05}
{'loss': 0.5125, 'learning_rate': 5.313390313390313e-06, 'epoch': 9.06}
{'loss': 0.2321, 'learning_rate': 5.277777777777778e-06, 'epoch': 9.06}
{'loss': 0.0441, 'learning_rate': 5.242165242165242e-06, 'epoch': 9.07}
{'loss': 0.0073, 'learning_rate

  0%|          | 0/101 [00:00<?, ?it/s]

{'eval_loss': 0.031560253351926804, 'eval_accuracy': 0.9938042131350682, 'eval_runtime': 16.7429, 'eval_samples_per_second': 48.199, 'eval_steps_per_second': 6.032, 'epoch': 10.0}
{'train_runtime': 2523.6136, 'train_samples_per_second': 24.711, 'train_steps_per_second': 3.091, 'train_loss': 0.4335261956774271, 'epoch': 10.0}


TrainOutput(global_step=7800, training_loss=0.4335261956774271, metrics={'train_runtime': 2523.6136, 'train_samples_per_second': 24.711, 'train_steps_per_second': 3.091, 'train_loss': 0.4335261956774271, 'epoch': 10.0})

In [22]:
kwargs = {
    "dataset_tags": "speech_commands",
    "dataset": "Speech Commands[no, go]",
    "model_name": f"{model_name}-finetuned-no-go-kws",
    "finetuned_from": model_id,
    "tasks": "audio-classification",
}

In [26]:
trainer.push_to_hub(**kwargs)

model.safetensors:   0%|          | 0.00/378M [00:00<?, ?B/s]

events.out.tfevents.1700163959.DESKTOP-MUPP1FQ.2272.0:   0%|          | 0.00/256k [00:00<?, ?B/s]

Upload 2 LFS files:   0%|          | 0/2 [00:00<?, ?it/s]

'https://huggingface.co/Teapack1/wav2vec2-base-960h-finetuned-no-go-kws/tree/main/'

## Inference

In [30]:
import sounddevice as sd
import soundfile as sf

duration = 2
sr = 16000

rec = sd.rec(int(duration) * sr, samplerate=sr, channels=1)
sd.wait()

sd.play(rec, samplerate=sr)

audio_file_path = 'recorded_audio.wav'
sf.write(audio_file_path, rec, sr)


In [31]:
from transformers import pipeline

classifier = pipeline("audio-classification", model="Teapack1/distilhubert-finetuned-no-go-kws")
classifier(audio_file_path)

[{'score': 0.9197768568992615, 'label': 'go'},
 {'score': 0.0801815614104271, 'label': 'no'},
 {'score': 2.5047777398867765e-06, 'label': 'backward'},
 {'score': 2.1814435058331583e-06, 'label': 'down'},
 {'score': 1.9438944036664907e-06, 'label': 'four'}]